In [63]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Evaluator, Pusher
from tfx.orchestration import pipeline
import tensorflow_data_validation as tfdv
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict
import pprint as pp
import os
from absl import logging

In [33]:
PIPELINE_NAME = "HAR-simple"
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
#METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

logging.set_verbosity(logging.INFO)

In [48]:
DATA_ROOT = os.path.join('../data/root/')
context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

In [49]:
example_gen = CsvExampleGen(input_base=DATA_ROOT)
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Processing input csv data ../data/root/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 6
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [50]:
artifact = example_gen.outputs['examples'].get()[0]
print(f'split names: {artifact.split_names}')
print(f'artifact uri: {artifact.uri}')

split names: ["train", "eval"]
artifact uri: pipelines/HAR-simple/CsvExampleGen/examples/6


In [52]:
train_uri = os.path.join(artifact.uri, 'Split-train')

tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [ ]:
def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    SOURCE: Akila Somasundaram at https://medium.com/data-science/tensorflow-transform-ensuring-seamless-data-preparation-in-production-99ffcf49f535
    '''

    # initialize an empty list
    records = []

    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):

        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()

        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()

        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)

        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))

        # append to the records list
        records.append(example_dict)

    return records

In [66]:
sample_records = get_records(dataset, 3)

# Print the output
pp.pprint(sample_records)

[{'features': {'feature': {'activity_id': {'bytesList': {'value': ['c2l0dGluZw==']}},
                           'ax': {'floatList': {'value': [-0.001]}},
                           'ay': {'floatList': {'value': [0.849]}},
                           'az': {'floatList': {'value': [0.48433334]}},
                           'participant_id': {'bytesList': {'value': ['Wk0=']}},
                           'timestamp': {'floatList': {'value': [1760380200.0]}}}}},
 {'features': {'feature': {'activity_id': {'bytesList': {'value': ['c2l0dGluZw==']}},
                           'ax': {'floatList': {'value': [-0.008]}},
                           'ay': {'floatList': {'value': [0.85]}},
                           'az': {'floatList': {'value': [0.483]}},
                           'participant_id': {'bytesList': {'value': ['Wk0=']}},
                           'timestamp': {'floatList': {'value': [1760380200.0]}}}}},
 {'features': {'feature': {'activity_id': {'bytesList': {'value': ['c2l0dGluZw==']

2025-12-02 18:34:51.323024: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [71]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 9
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [72]:
context.show(statistics_gen.outputs['statistics'])

In [73]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )
context.run(schema_gen)
context.show(schema_gen.outputs['schema'])

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


,Type,Presence,Valency,Domain
Feature name,,,,
'activity_id',STRING,required,,'activity_id'
'ax',FLOAT,required,,-
'ay',FLOAT,required,,-
'az',FLOAT,required,,-
'participant_id',STRING,required,,'participant_id'
'timestamp',FLOAT,required,,-


,Values
Domain,
'activity_id',"'lying', 'running', 'sitting', 'walking'"
'participant_id',"'LL', 'VV', 'YL', 'YT', 'ZM'"


In [ ]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])

context.run(example_validator)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


In [ ]:
context.show(example_validator.outputs['anomalies'])